In [1]:
using Symbolics

function HM(sz::Int...; symbol::Symbol)
    (@variables $symbol[map(x -> 1:x, sz)...])[1]
end

function HM(sz::AbstractRange{Int}...; symbol::Symbol)
    (@variables $symbol[sz...])[1]
end

eltypes(::Type{Symbolics.Arr{T,D}}) where {T,D} = (T, D)
eltypes(x::Symbolics.Arr) = eltypes(typeof(x))

Base.similar(A::AbstractArray, T::Type, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(A, T, tuple(map(x -> length(x), dims)...))
    
Base.similar(f::Union{Function,DataType}, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(f, tuple(map(x -> length(x), dims)...))

function printHM(Hx::Symbolics.Arr{T,2}) where {T}
    println("2 method called! ", Hx)
    if any(firstindex.([Hx],[1:2...]') .!= 1)
        r,c = size(Hx)
        [Hx[i+r*j] for i in 1:r, j in 0:(c-1)]
    else
        [x for x in Hx]
    end
end

function printHM(Hx::Symbolics.Arr{T,D}, dim1=0, dim2=1, dpths=nothing) where {T,D}
    println("general method called! ", Hx)
    if any(firstindex.([Hx],[1:D...]') .!= 1)
        [Hx[i] for i in CartesianIndex(repeat([1],D)...):CartesianIndex(size(Hx)...)]
    else
        [x for x in Hx]
    end
end

printHM (generic function with 5 methods)

In [2]:
sz = 3; HV = HM(repeat([sz],sz)..., symbol=:v)
printHM(HV)

general method called! v[1:3,1:3,1:3]


3×3×3 Array{Num, 3}:
[:, :, 1] =
 v[1, 1, 1]  v[1, 2, 1]  v[1, 3, 1]
 v[2, 1, 1]  v[2, 2, 1]  v[2, 3, 1]
 v[3, 1, 1]  v[3, 2, 1]  v[3, 3, 1]

[:, :, 2] =
 v[1, 1, 2]  v[1, 2, 2]  v[1, 3, 2]
 v[2, 1, 2]  v[2, 2, 2]  v[2, 3, 2]
 v[3, 1, 2]  v[3, 2, 2]  v[3, 3, 2]

[:, :, 3] =
 v[1, 1, 3]  v[1, 2, 3]  v[1, 3, 3]
 v[2, 1, 3]  v[2, 2, 3]  v[2, 3, 3]
 v[3, 1, 3]  v[3, 2, 3]  v[3, 3, 3]

In [3]:
arguments(Symbolics.value(HV[1,1,2]))[2:end]

3-element Vector{Any}:
 1
 1
 2

In [4]:
function lex(sz, f) # No equivalent to Iterable in Julia...so this will remain generic
    # it, terms = [f] .|> [enumerate length]
    sum(enumerate(f)) do (i, x)
        (x-1) * sz^(i-1)
    end
end

symbolicarray_fromsymbol(f) = arguments(Symbolics.value(f))[1]

list_fromsymbol(f)::Vector = arguments(Symbolics.value(f))[2:end]

function lex(sz, f::Num)
    lex(sz, list_fromsymbol(f))
end

lex.(sz, printHM(HV))

general method called! v[1:3,1:3,1:3]


3×3×3 Array{Int64, 3}:
[:, :, 1] =
 0  3  6
 1  4  7
 2  5  8

[:, :, 2] =
  9  12  15
 10  13  16
 11  14  17

[:, :, 3] =
 18  21  24
 19  22  25
 20  23  26

In [180]:
import Base.∘

# (compose(fi::T...)::T) where {T} = reduce(∘, fi) # This is where the type system breaks...

compose(f1::Vector, f2::Vector)::Vector = map(x->f1[x],f2)

∘(f1::Vector, f2::Vector)::Vector = compose(f1, f2)

vector_function(fi::Vector)::Function = x -> fi[x]

function lazy_compose_1(fi::Vector...)::Vector
    fcomposed = mapreduce(vector_function, ∘, fi)
    [fcomposed(i) for i in 1:length(fi[1])]
end

eager_compose(fi::Num...)::Num = symbolicarray_fromsymbol(fi[1])[compose((fi .|> list_fromsymbol)...)...]

function lazy_compose(fi::Num...)::Num
    fcomposed = compose((fi .|> list_fromsymbol .|> (fi::Vector -> (x -> fi[x])))...)
    symbolicarray_fromsymbol(fi[1])[(fcomposed(i) for i in 1:length(list_fromsymbol(fi[1])))...]
end

compose(fi::Num...)::Num = eager_compose(fi...)

# Composition is associative
# compose(f1::T, fi::T...) where {T} = compose(compose(f1,f2),fi...)

# function compose(f1::Num, fi::Num...)
#     HM = symbolicarray_fromsymbol(f1)
#     HM[compose(([f1, fi...] .|> list_fromsymbol)...)...]
# end

# f1list, f2list = [f1, f2] .|> list_fromsymbol'

compose (generic function with 4 methods)

In [179]:
compose.(HV[2,3,1],[HV[2,1,3], HV[1,3,2]])

2-element Vector{Num}:
 v[3, 2, 1]
 v[2, 1, 3]

In [175]:
compose(HV[2,3,1], HV[2,1,3], HV[1,3,2])

v[3, 1, 2]

In [151]:
lazy_compose(HV[2,3,1],HV[2,1,3])

v[3, 2, 1]

In [29]:
function genrandfunc(sz)
    history = []
    function randFunc(ctx)
        f = rand(1:sz, sz)
        push!(history, (f,ctx))
        f
    end
    randFunc, ()->history
end
randFunc, getHistory = genrandfunc(sz);

In [30]:
compose(HV[randFunc("cmpSym")...], HV[randFunc("cmpSym")...])

v[2, 3, 3]

In [31]:
getHistory()

2-element Vector{Any}:
 ([3, 3, 2], "cmpSym")
 ([3, 1, 2], "cmpSym")

In [32]:
list_fromlex(sz, terms, lex) = [((lex ÷ (sz ^ i)) % sz) + 1 for i in 0:(terms-1)]

symbol_fromlex(HM, sz, terms, lex) = HM[list_fromlex(sz, terms, lex)...]

lex_identfunc(sz) = lex(sz, 1:sz)

symbol_fromlex(HV, sz, sz, lex_identfunc(sz))

v[1, 2, 3]

In [177]:
test_f1 = [2,1,3]

@code_typed lazy_compose(HV[2,3,1],HV[2,1,3],HV[1,2,3])

CodeInfo(
1 ─ %1  = Main.Num::Type{Num}
│   %2  = Base.getfield(fi, 1, false)::Num
│   %3  = invoke list_fromsymbol(%2::Num)::Vector
│   %4  = Main.:(var"#275#278")::Type{var"#275#278"}
│   %5  = Core.typeof(%3)::Type{Vector{T}} where T
│   %6  = Core.apply_type(%4, %5)::Type{var"#275#278"{_A}} where _A
│   %7  = %new(%6, %3)::var"#275#278"
│   %8  = Base.getfield(fi, 2, false)::Num
│   %9  = invoke list_fromsymbol(%8::Num)::Vector
│   %10 = Main.:(var"#275#278")::Type{var"#275#278"}
│   %11 = Core.typeof(%9)::Type{Vector{T}} where T
│   %12 = Core.apply_type(%10, %11)::Type{var"#275#278"{_A}} where _A
│   %13 = %new(%12, %9)::var"#275#278"
│   %14 = Base.getfield(fi, 3, false)::Num
│   %15 = invoke list_fromsymbol(%14::Num)::Vector
│   %16 = Main.:(var"#275#278")::Type{var"#275#278"}
│   %17 = Core.typeof(%15)::Type{Vector{T}} where T
│   %18 = Core.apply_type(%16, %17)::Type{var"#275#278"{_A}} where _A
│   %19 = %new(%18, %15)::var"#275#278"
│   %20 = Main.compose(%7, %13, %19)::Any
